
# Introduction :
L'objectif de ce notebook est de réaliser une analyse exploratoire des données (EDA) sur un jeu de données d'analyse des sentiments. Cette analyse vise à comprendre les sentiments (positif, négatif, neutre) exprimés dans un texte. L'EDA est essentielle pour :

Comprendre la distribution des sentiments dans le jeu de données.
Identifier les problèmes de qualité des données, tels que les valeurs manquantes ou les doublons.
Déterminer les étapes de prétraitement nécessaires, comme la suppression des stopwords et la lemmatisation.
Analyser les mots clés pour mieux comprendre les caractéristiques textuelles des différents sentiments.




## Importer les bibliothèques nécessaires

In [1]:
# Importation de la bibliothèque pandas pour la manipulation des données
import pandas as pd

# Importation de defaultdict de la bibliothèque collections pour créer des dictionnaires avec des valeurs par défaut
from collections import defaultdict

# Cette ligne est utilisée dans les notebooks Jupyter pour afficher les graphiques directement dans le notebook
%matplotlib inline

# Importation des bibliothèques Plotly pour la visualisation interactive des données
from plotly import graph_objs as go  # Importation de graph_objs pour créer des graphiques personnalisés
import plotly.express as px  # Importation de plotly.express pour des visualisations plus simples et rapides

# Importation de la bibliothèque nltk (Natural Language Toolkit) pour le traitement du langage naturel (NLP)
import nltk

# Importation du module string pour travailler avec des chaînes de caractères, en particulier pour les ponctuations
import string

# Importation du module stopwords de NLTK pour les mots vides courants (comme "et", "le", "la", etc.) à supprimer dans les textes
from nltk.corpus import stopwords

# Importation du PorterStemmer de NLTK pour effectuer du stemming sur les mots (réduire les mots à leur racine)
from nltk.stem import PorterStemmer

# Redondance : Importation à nouveau de defaultdict. Cela n'est pas nécessaire car il a déjà été importé plus haut.
from collections import defaultdict

# Téléchargement des ressources nécessaires si elles ne sont pas déjà présentes sur la machine
nltk.download('punkt')  # Télécharge les ressources nécessaires pour la tokenisation des mots (découper le texte en mots)
nltk.download("stopwords")  # Télécharge la liste des mots vides (stopwords) qui sont souvent ignorés lors du prétraitement des textes


[nltk_data] Downloading package punkt to C:\Users\WIJDANE
[nltk_data]     TAFTAF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\WIJDANE
[nltk_data]     TAFTAF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Charger les données.

In [2]:
# Charger les données depuis un fichier CSV
csv_file_path = 'train.csv'  # Spécifie le chemin du fichier CSV à charger
df = pd.read_csv(csv_file_path)  # Utilise pandas pour lire le fichier CSV et le charger dans un DataFrame

# Supprimer la colonne 'selected_text' car elle n'est pas nécessaire pour cette analyse
df = df.drop(columns=['selected_text'])  # Cette ligne supprime la colonne 'selected_text' du DataFrame
# Cette colonne peut contenir des informations supplémentaires non pertinentes pour l'analyse des sentiments

# Convertir la colonne 'text' en type chaîne de caractères (string)
df["text"] = df["text"].astype(str)  # Cette ligne s'assure que toutes les valeurs de la colonne 'text' sont de type string
# Cela est important car parfois certaines valeurs peuvent être de type numérique ou d'autres types, ce qui pourrait poser problème lors du traitement du texte.


## Exploration des données
Afficher des informations de base sur les données en utilisant df.shape, df.info(), et df.head()

In [3]:
# Afficher la forme et les informations de base du DataFrame
print(df.shape)

(27664, 3)


In [4]:
# Affiche des informations détaillées sur le DataFrame, telles que la forme du DataFrame (nombre de lignes et de colonnes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27664 entries, 0 to 27663
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     27664 non-null  object
 1   text       27664 non-null  object
 2   sentiment  27664 non-null  object
dtypes: object(3)
memory usage: 648.5+ KB


## Visualisation de la distribution des sentiments :

Regrouper et compter les valeurs de sentiment.

In [5]:
# Grouper les données par sentiment et compter le nombre d'entrées pour chaque sentiment
temp = df.groupby('sentiment').count()['text'].reset_index().sort_values(by='text', ascending=False)

# Appliquer un style de dégradé de fond à la colonne 'temp' en utilisant la palette de couleurs 'Purples'
styled_temp = temp.style.background_gradient(cmap='Purples')



Visualiser la distribution des sentiments à l'aide d'un graphique en entonnoir.


In [10]:
# Créer un Funnel-Chart pour visualiser la distribution des sentiments
fig = go.Figure(go.Funnelarea(
    text=temp.sentiment,  # Les étiquettes des catégories (sentiment)
    values=temp.text,     # Les valeurs (nombre de textes) pour chaque catégorie
    title={"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}  # Titre du graphique avec une position centrale
))

In [11]:
# Afficher les premières lignes du DataFrame
df.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


## Prétraitement du texte :

Définir la fonction preprocess_tweet.

In [12]:
# Fonction de prétraitement pour nettoyer et tokeniser les tweets
def preprocess_tweet(tweet):
    tweet = tweet.lower()  # Convertir le texte en minuscules
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))  # Supprimer la ponctuation
    tokens = nltk.word_tokenize(tweet)  # Tokeniser le texte (diviser le texte en mots)
    stemmer = PorterStemmer()  # Créer une instance du stemmer Porter (pour le stemming des mots)
    stopwords_set = set(stopwords.words("english"))  # Obtenir un ensemble de mots vides (stopwords) en anglais
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords_set]  # Appliquer le stemming et supprimer les stopwords
    return tokens  # Retourner les tokens traités

1. Définir la fonction calculate_word_counts.
2. Tokeniser et prétraiter les données textuelles en utilisant NLTK.

In [13]:
# Fonction pour calculer le nombre d'occurrences de chaque mot dans une liste de tweets
def calculate_word_counts(tweets):
    word_count = defaultdict(int)  # Initialiser un dictionnaire avec une valeur par défaut de 0
    for tweet in tweets:
        tokens = preprocess_tweet(tweet)  # Prétraiter chaque tweet pour obtenir les tokens
        for token in tokens:
            word_count[token] += 1  # Incrémenter le compteur pour chaque token trouvé
    return word_count  # Retourner le dictionnaire contenant les comptages des mots

## Analyse du nombre de mots :

1. Calculer le nombre de mots pour chaque classe de sentiment.
2. Convertir les comptes de mots en DataFrames.
3. Afficher les mots les plus fréquents et leurs occurrences pour chaque classe de sentiment.

In [14]:
# Calculer le nombre de mots pour les tweets ayant un sentiment 'positif'
word_count_positive = calculate_word_counts(df[df['sentiment'] == 'positive']['text'])

# Calculer le nombre de mots pour les tweets ayant un sentiment 'négatif'
word_count_negative = calculate_word_counts(df[df['sentiment'] == 'negative']['text'])

# Calculer le nombre de mots pour les tweets ayant un sentiment 'neutre'
word_count_neutral = calculate_word_counts(df[df['sentiment'] == 'neutral']['text'])

In [15]:
# Convertir les comptages de mots en DataFrames pour une meilleure visualisation
word_count_positive_df = pd.DataFrame(word_count_positive.items(), columns=['Word', 'Count'])
word_count_negative_df = pd.DataFrame(word_count_negative.items(), columns=['Word', 'Count'])
word_count_neutral_df = pd.DataFrame(word_count_neutral.items(), columns=['Word', 'Count'])


In [16]:
# Afficher les mots les plus fréquents et leurs comptages pour chaque classe de sentiment

# Afficher les comptages des mots pour les tweets avec un sentiment positif
print("Word Counts - Positive Sentiment:")
print(word_count_positive_df.head())  # Affiche les 5 premiers mots et leur fréquence pour le sentiment positif

# Afficher les comptages des mots pour les tweets avec un sentiment négatif
print("\nWord Counts - Negative Sentiment:")
print(word_count_negative_df.head())  # Affiche les 5 premiers mots et leur fréquence pour le sentiment négatif

# Afficher les comptages des mots pour les tweets avec un sentiment neutre
print("\nWord Counts - Neutral Sentiment:")
print(word_count_neutral_df.head())  # Affiche les 5 premiers mots et leur fréquence pour le sentiment neutre


Word Counts - Positive Sentiment:
    Word  Count
0    2am      2
1   feed      9
2   babi     65
3    fun    344
4  smile     45

Word Counts - Negative Sentiment:
    Word  Count
0   sooo     38
1    sad    398
2   miss    660
3    san      9
4  diego      6

Word Counts - Neutral Sentiment:
                       Word  Count
0                        id     72
1                   respond     11
2                        go   1055
3  httpwwwdothebouncycomsmf      1
4                 shameless      1


## Visualisation des mots les plus fréquents :

1. Afficher les mots les plus fréquents pour le sentiment positif avec un DataFrame stylisé.
2. Visualiser les mots positifs les plus fréquents à l'aide d'un treemap.
3. Répéter les étapes ci-dessus pour les sentiments négatif et neutre.

In [17]:
# Trier et sélectionner les 20 mots les plus fréquents pour chaque classe de sentiment

# Pour les tweets avec un sentiment positif, trier les mots par fréquence et sélectionner les 20 premiers
top_positive_words = word_count_positive_df.sort_values(by='Count', ascending=False).head(20)

# Pour les tweets avec un sentiment négatif, trier les mots par fréquence et sélectionner les 20 premiers
top_negative_words = word_count_negative_df.sort_values(by='Count', ascending=False).head(20)

# Pour les tweets avec un sentiment neutre, trier les mots par fréquence et sélectionner les 20 premiers
top_neutral_words = word_count_neutral_df.sort_values(by='Count', ascending=False).head(20)


In [18]:
# Afficher les 20 mots les plus fréquents pour les tweets ayant un sentiment positif
print("Top 20 Words - Positive Sentiment:")
print()  # Ajouter une ligne vide pour mieux organiser la sortie

# Appliquer un dégradé de fond pour styliser le DataFrame et l'afficher
styled_positive_words = top_positive_words.style.background_gradient(cmap='Blues')

# Afficher le DataFrame stylisé avec le dégradé de fond
styled_positive_words


Top 20 Words - Positive Sentiment:



,Word,Count
74,day,1326
16,love,1139
103,good,1056
66,happi,852
112,thank,814
38,im,742
143,mother,668
49,go,573
55,hope,519
426,great,481


In [20]:
# Créer un treemap avec Plotly Express pour visualiser les mots positifs les plus fréquents
fig = px.treemap(top_positive_words, path=['Word'], values='Count', title='Tree Of Unique Positive Words')

In [21]:
# Afficher les 20 mots les plus fréquents pour les tweets ayant un sentiment négatif
print("\nTop 20 Words - Negative Sentiment:")
print()  # Ajouter une ligne vide pour mieux organiser la sortie

# Appliquer un dégradé de fond pour styliser le DataFrame et l'afficher
styled_negative_words = top_negative_words.style.background_gradient(cmap='Purples')

# Afficher le DataFrame stylisé avec le dégradé de fond
styled_negative_words


Top 20 Words - Negative Sentiment:



,Word,Count
54,im,1227
22,go,735
2,miss,660
155,get,613
66,work,493
102,like,492
106,dont,469
199,feel,467
299,cant,463
39,day,404


In [22]:
# Créer un treemap avec Plotly Express pour visualiser les mots négatifs les plus fréquents
fig = px.treemap(top_negative_words, path=['Word'], values='Count', title='Tree Of Unique Negative Words')


In [23]:
# Afficher les 20 mots les plus fréquents pour les tweets ayant un sentiment neutre
print("\nTop 20 Words - Neutral Sentiment:")
print()  # Ajouter une ligne vide pour mieux organiser la sortie

# Appliquer un dégradé de fond pour styliser le DataFrame et l'afficher
styled_neutral_words = top_neutral_words.style.background_gradient(cmap='Greens')

# Afficher le DataFrame stylisé avec le dégradé de fond
styled_neutral_words


Top 20 Words - Neutral Sentiment:



,Word,Count
19,im,1058
2,go,1055
23,get,819
164,work,647
320,day,638
42,got,539
285,dont,491
112,like,482
224,time,468
233,lol,455


In [24]:
# Créer un treemap avec Plotly Express pour visualiser les mots neutres les plus fréquents
fig = px.treemap(top_neutral_words, path=['Word'], values='Count', title='Tree Of Unique Neutral Words')